# Notebook to manually inspect genes/transcripts<br>


This is a text cell. Start editing!




In [1]:
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import group.research.aging.spark.extensions.functions.ConcatenateString
import group.research.aging.spark.extensions.functions.Concatenate
import ammonite.ops._
import ammonite.ops.ImplicitWd._
import org.apache.spark.storage.StorageLevel

In [2]:
import better.files._
import File._
import java.io.{File => JFile}

In [49]:
val data_path = "/data/"
val samples_path = s"${data_path}samples/"
val ensembl_path = s"${data_path}ensembl/99/"
val graphdb_path = s"${data_path}databases/graphdb/import"
val yspecies = s"${data_path}sources/yspecies/"
val ranked_path = yspecies + "data/output/intersections/intersections_ranked.tsv"
val stage_one_input = yspecies +  "data/interim/selected/lifespan/"
val stage_two_input = yspecies +  "data/interim/stage_2/input/"

In [54]:
val not_null = udf[String, String](str=> if(str==null || str=="\\N") "" else str)
val underscored =udf[String, String](str=> str.replace(" ", "_"))
def undot(s: String): String = {  s.lastIndexOf(".") match { case -1 => s case i => s.substring(0,i)} }
val u_undot = udf[String, String](undot _)

## Load transcript maps




In [8]:
//val anage = spark.readTSV("/data/databases/anage/anage_data.tsv", header = true).withColumn("scientific_name", concat($"Genus", lit(" "), $"Species"))
//anage.show()
//val anage_animals = spark.readTSV("/data/ensembl/99/view_animals_anage.tsv", header = true).withColumnRenamed("latin_name", "scientific_name")
//anage_animals.show(10,10000)
val species = spark.readTSV(ensembl_path + "ensembl_anage_vertebrates.tsv", header = true).withColumnRenamed("latin_name", "scientific_name")
species.show(10,10000)

+--------------------+--------------------------------+-----------+-----------+------------------+-----------------------+-----+-----+------+-------+----------------+-----------+--------------+-----------+--------------+--------------------+------------------+-------+-----------+----------------+--------------------+--------------+----------------+--------------+-----------+
|         common_name|                 scientific_name|taxonomy_id|   assembly|assembly_accession|              genebuild|class|order|family|species|maximum_lifespan|body_mass_g|metabolic_rate|temperature|gestation_days|female_maturity_days|male_maturity_days|weaning|litter_size|litters_per_year|inter_birth_interval|birth_weight_g|weaning_weight_g|adult_weight_g|growth_rate|
+--------------------+--------------------------------+-----------+-----------+------------------+-----------------------+-----+-----+------+-------+----------------+-----------+--------------+-----------+--------------+--------------------+---

In [9]:
val path = File(s"${ensembl_path}species")
val tx2gene = (
  for(fl <- path.children.flatMap(_.children
    .filter(c=>c.name.endsWith(".gtf")&& !c.name.contains("abinitio")&& !c.name.contains("hapl_scaff")))
    ) yield fl.parent.name.capitalize -> spark.readTSV(fl.pathAsString.replace(".gtf", "_tx2gene.tsv")).toDF("transcript", "gene")
  ).toMap[String, DataFrame]

val h = tx2gene.head
println(h._1)
h._2.show(10,1000)

Otolemur_garnettii
+------------------+------------------+
|        transcript|              gene|
+------------------+------------------+
|ENSOGAT00000012158|ENSOGAG00000012155|
|ENSOGAT00000031338|ENSOGAG00000033509|
|ENSOGAT00000026645|ENSOGAG00000030153|
|ENSOGAT00000031633|ENSOGAG00000034131|
|ENSOGAT00000030922|ENSOGAG00000033908|
|ENSOGAT00000015494|ENSOGAG00000015488|
|ENSOGAT00000030131|ENSOGAG00000029906|
|ENSOGAT00000004627|ENSOGAG00000004624|
|ENSOGAT00000031760|ENSOGAG00000029208|
|ENSOGAT00000029851|ENSOGAG00000032266|
+------------------+------------------+
only showing top 10 rows



## Load samples and transcript expressions




In [13]:
val s = spark.readTSV(s"${graphdb_path}/transcript_stable_expressions.tsv", header = true).as[(String, String, String, Double)]
transcript_expressions.show(10)

+----------+------------------+--------------------+---------+
|       run| transcript_stable|          transcript|      TPM|
+----------+------------------+--------------------+---------+
|SRR2040645|ENSCCET00000000012|ENSCCET00000000012.1|15.003977|
|SRR2040645|ENSCCET00000000003|ENSCCET00000000003.1|75.039963|
|SRR2040645|ENSCCET00000000007|ENSCCET00000000007.1| 0.203121|
|SRR2040645|ENSCCET00000000047|ENSCCET00000000047.1|77.176479|
|SRR2040645|ENSCCET00000000005|ENSCCET00000000005.1|      0.0|
|SRR2040645|ENSCCET00000000092|ENSCCET00000000092.1|      0.0|
|SRR2040645|ENSCCET00000000174|ENSCCET00000000174.1| 0.771706|
|SRR2040645|ENSCCET00000000010|ENSCCET00000000010.1|      0.0|
|SRR2040645|ENSCCET00000000154|ENSCCET00000000154.1| 0.038792|
|SRR2040645|ENSCCET00000000006|ENSCCET00000000006.1|72.543704|
+----------+------------------+--------------------+---------+
only showing top 10 rows



In [57]:
//val transcripts_to_genes = tx2gene.map{ case (key, value) => value.withColumn("species", lit(key))}.reduce(_.union(_))
//transcripts_to_genes.select("species", "gene", "transcript").writeTSV(s"${graphdb_path}/transcripts_to_genes.tsv")
val transcripts_to_genes = spark.readTSV(s"${graphdb_path}/transcripts_to_genes.tsv", true).as[(String, String, String)]
transcripts_to_genes.show(10)

+------------------+------------------+------------------+
|           species|              gene|        transcript|
+------------------+------------------+------------------+
|Otolemur_garnettii|ENSOGAG00000012155|ENSOGAT00000012158|
|Otolemur_garnettii|ENSOGAG00000033509|ENSOGAT00000031338|
|Otolemur_garnettii|ENSOGAG00000030153|ENSOGAT00000026645|
|Otolemur_garnettii|ENSOGAG00000034131|ENSOGAT00000031633|
|Otolemur_garnettii|ENSOGAG00000033908|ENSOGAT00000030922|
|Otolemur_garnettii|ENSOGAG00000015488|ENSOGAT00000015494|
|Otolemur_garnettii|ENSOGAG00000029906|ENSOGAT00000030131|
|Otolemur_garnettii|ENSOGAG00000004624|ENSOGAT00000004627|
|Otolemur_garnettii|ENSOGAG00000029208|ENSOGAT00000031760|
|Otolemur_garnettii|ENSOGAG00000032266|ENSOGAT00000029851|
+------------------+------------------+------------------+
only showing top 10 rows



In [82]:
transcript_expressions.select($"run", $"transcript_stable".as("transcript"), $"TPM")
  .join(transcripts_to_genes, Seq("transcript"))
  .select("run", "species", "gene", "transcript", "TPM")
  .writeTSV(s"${graphdb_path}/transcript_extended_expressions.tsv", header = true)
val extended_expressions = spark.readTSV(s"${graphdb_path}/transcript_extended_expressions.tsv", true)
extended_expressions.show(10)

parts of /data/databases/graphdb/import/transcript_extended_expressions.tsv merged!


org.apache.spark.sql.AnalysisException: Path does not exist: file:/opt/polynote/${graphdb_path}/transcript_extended_expressions.tsv;

## Experimenting with transcripts




In [34]:
val intersections_path = yspecies + "data/output/intersections/"
val ranked_path =intersections_path + "intersections_ranked.tsv"
val stage_one_input = yspecies +  "data/interim/selected/lifespan/"
val stage_two_input = yspecies +  "data/interim/stage_2/input/"
val genes_ranked = spark.readTSV(ranked_path, true)
genes_ranked.show(10, 100)

+---------------+--------+----+---------+------------+---------+----------------+--------------+------------------+-----------+-----------+-------+-------------------+-------------------+---------+-------------------+------------+-----------+------------------------+------------------------------+---------------------------------+--------------------------+----------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
| reference_gene|  symbol|rank|ranks_sum|repeats_rank|shap_rank|kendall_tau_rank|frequency_rank|tissue_linear_rank|genage_rank|repeats_all|repeats|      mean_abs_shap|   mean_kendall_tau|frequency|      max_linear_r2|genage_count|  direction|repeats_lifespan_stage_1|mean_abs_shap_lifespan_stage_1|mean_kendall_tau_lifespan_stage_1|direction_lifespan_stage_1|                                                               other_life_history_traits|  

In [37]:
kernel.display.html(File(stage_two_input).children.toList.mkString("<br>"))

/data/sources/yspecies/data/interim/stage_2/input/genes_meta.tsv /data/sources/yspecies/data/interim/stage_2/input/expressions.tsv /data/sources/yspecies/data/interim/stage_2/input/genes.tsv /data/sources/yspecies/data/interim/stage_2/input/samples.tsv /data/sources/yspecies/data/interim/stage_2/input/species.tsv /data/sources/yspecies/data/interim/stage_2/input/.~lock.expressions.tsv#

In [76]:
case class ExpressionDataset(samples: DataFrame, 
  genes: DataFrame, 
  genes_meta: Dataset[(String, String, String)], 
  species: DataFrame, 
  expressions: DataFrame
  ){
    def get_genes_by_symbol(symbol: String): DataFrame = {
      val id = this.genes_meta.where($"symbol"==="NOXA1").first()._1
      this.get_genes_by_ensembl_id(id)
    }

    def get_genes_by_ensembl_id(id: String): DataFrame= {      
      val geneRow = this.genes.where($"Homo_sapiens" === id).first()
      val rowMap:Map[String, String] = geneRow.getValuesMap(this.genes.columns)
      rowMap.flatMap{ 
          case (k, v) if v == null => List.empty[(String, String)]
          case (k, v) => v.split(";").map(g=>g->k)
          case _ => List.empty[(String, String)]
      }.toSeq.toDF("gene", "species")
        .join(this.species.select("species", "common_name", "lifespan", "order", "family"), Seq("species"))
        .orderBy($"lifespan".desc)
    }
  }
object ExpressionDataset{
    def apply(path: String): ExpressionDataset = {
       val stage = File(path).children.toList.map(c=> c.name->c.pathAsString).toMap[String, String]
       ExpressionDataset(
           spark.readTSV(stage("samples.tsv"), true), 
           spark.readTSV(stage("genes.tsv"), true), 
           spark.readTSV(stage("genes_meta.tsv"), true).as[(String, String, String)], 
           spark.readTSV(stage("species.tsv"), true), 
           spark.readTSV(stage("expressions.tsv"), true)
        )

    }
}


In [46]:
val stage_one = ExpressionDataset(stage_one_input)
stage_one

ExpressionDataset([run: string, bioproject: string ... 37 more fields],[Homo_sapiens: string, Ailuropoda_melanoleuca: string ... 36 more fields],[Homo_sapiens: string, species: string ... 1 more field],[species: string, common_name: string ... 17 more fi

In [41]:
val stage_two = ExpressionDataset(stage_two_input)
stage_two

ExpressionDataset([run: string, bioproject: string ... 37 more fields],[Homo_sapiens: string, Ailuropoda_melanoleuca: string ... 36 more fields],[Homo_sapiens: string, species: string ... 1 more field],[species: string, common_name: string ... 17 more fi

In [75]:
val noxa = stage_two.get_genes_by_symbol("NOXA1")
val joined = noxa.join(extended_expressions, Seq("species", "gene"))

import  org.apache.spark.sql.expressions.Window

val sums = joined
  .groupBy($"species", $"common_name", $"lifespan", $"gene", $"transcript")
  .agg(sum($"TPM").as("TPM_sum"))

val best = sums
  .withColumn("max_TPM_sum",max($"TPM_sum").over(Window.partitionBy($"species", $"common_name", $"lifespan", $"gene")))
  .where($"max_TPM_sum" === $"TPM_sum")
  .drop($"max_TPM_sum").drop($"TPM_sum").orderBy($"lifespan".desc)
best.show(10)
  //.first()

+--------------------+--------------------+--------+------------------+------------------+
|             species|         common_name|lifespan|              gene|        transcript|
+--------------------+--------------------+--------+------------------+------------------+
|        Homo_sapiens|               Human|   122.5|   ENSG00000188747|   ENST00000341349|
|     Gorilla_gorilla|             Gorilla|    60.1|ENSGGOG00000041617|ENSGGOT00000058456|
|     Pan_troglodytes|          Chimpanzee|    59.4|ENSPTRG00000042665|ENSPTRT00000086905|
|      Equus_caballus|               Horse|    57.0|ENSECAG00000018904|ENSECAT00000020347|
|        Pan_paniscus|Pygmy chimpanzee ...|    55.0|ENSPPAG00000037345|ENSPPAT00000051482|
|      Macaca_mulatta|       Rhesus monkey|    40.0|ENSMMUG00000019602|ENSMMUT00000027525|
| Macaca_fascicularis| Long-tailed macaque|    39.0|ENSMFAG00000030739|ENSMFAT00000065589|
|   Macaca_nemestrina|     Pigtail macaque|    37.6|ENSMNEG00000035523|ENSMNET00000048160|

In [80]:
extended_expressions.show(10)

+----------+-------------------+------------------+------------------+---------+
|       run|            species|              gene|        transcript|      TPM|
+----------+-------------------+------------------+------------------+---------+
|SRR5039773|Anolis_carolinensis|ENSACAG00000000015|ENSACAT00000000015| 7.415931|
|SRR5039741|Anolis_carolinensis|ENSACAG00000000015|ENSACAT00000000015|10.039286|
| SRR579557|Anolis_carolinensis|ENSACAG00000000015|ENSACAT00000000015|18.844585|
|SRR5039773|Anolis_carolinensis|ENSACAG00000000246|ENSACAT00000000233| 0.150917|
|SRR5039741|Anolis_carolinensis|ENSACAG00000000246|ENSACAT00000000233| 0.147861|
| SRR579557|Anolis_carolinensis|ENSACAG00000000246|ENSACAT00000000233|  0.45184|
|SRR5039773|Anolis_carolinensis|ENSACAG00000000860|ENSACAT00000000788|  2.60776|
|SRR5039741|Anolis_carolinensis|ENSACAG00000000860|ENSACAT00000000788| 4.374273|
| SRR579557|Anolis_carolinensis|ENSACAG00000000860|ENSACAT00000000788| 9.545411|
|SRR5039773|Anolis_carolinen

In [84]:
extended_expressions

### Loading proteins data<br>




In [68]:
val mapping_cols = Seq("UniProtKB-AC","UniProtKB-ID","Entrez","RefSeq","GI","PDB","GO",
"UniRef100","UniRef90","UniRef50","UniParc","PIR",
"NCBI-taxon","MIM","UniGene","PubMed",
"EMBL","EMBL-CDS","Ensembl","Ensembl_TRS","Ensembl_PRO","Additional PubMed")

In [72]:
val uniprot_mapping = spark.readTSV("/data/indexes/uniprot/2020_05/idmapping_selected.tab").toDF(mapping_cols:_*)
uniprot_mapping.limit(20).show(20, 1000)

+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|UniProtKB-AC|UniProtKB-ID| Entrez|     RefSeq|                             GI| PDB|                                GO|       UniRef100|       UniRef90|       UniRef50|      UniParc| PIR|NCBI-taxon| MIM|UniGene|            PubMed|    EMBL|  EMBL-CDS|Ensembl|Ensembl_TRS|Ensembl_PRO|Additional PubMed|
+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|      Q6GZX4|  001R_FRG3G|2947773|YP_031579.1|             81941549; 49237298|null|             

In [45]:
case class TranscriptExpressionDataset(
  dataset: ExpressionDataset, 
  extended_expressions: DataFrame, 
  uniprot_mapping: DataFrame )
{

  lazy val transcript2uniprot = uniprot_mapping
    .select($"Ensembl_TRS".as("transcript"), $"UniProtKB-AC".as("uniprot"))
    .where($"transcript".isNotNull)

  def transcripts_to_isoform(df: DataFrame) = {
    val joined = df.join(extended_expressions, Seq("species", "gene"))

    import  org.apache.spark.sql.expressions.Window

    val sums = joined
      .groupBy($"species", $"common_name", $"lifespan", $"gene", $"transcript")
      .agg(sum($"TPM").as("TPM_sum"))

    val best = sums
      .withColumn("max_TPM_sum",max($"TPM_sum").over(Window.partitionBy($"species", $"common_name", $"lifespan", $"gene")))
      .where($"max_TPM_sum" === $"TPM_sum")
      .drop($"max_TPM_sum").drop($"TPM_sum").orderBy($"lifespan".desc)
    best
  }

  def best_protein_by_ensembl_id(id: String) = {
    val df = transcripts_to_isoform(this.dataset.get_genes_by_ensembl_id(id))
    val proteins = df.join(transcript2uniprot, Seq("transcript"),"left").orderBy($"lifespan".desc)
    //print(df.count, proteins.count)
    proteins.cache()
  }

  def best_protein_by_symbol(symbol: String) = {
    val df = transcripts_to_isoform(this.dataset.get_genes_by_symbol(symbol))
    val proteins = df.join(transcript2uniprot, Seq("transcript"), "left").orderBy($"lifespan".desc)
    //print(df.count, proteins.count)
    proteins.cache()
  }
  
}



In [73]:
val stage_two_exp = TranscriptExpressionDataset(stage_two, extended_expressions, uniprot_mapping)
stage_two_exp

TranscriptExpressionDataset(ExpressionDataset([run: string, bioproject: string ... 37 more fields],[Homo_sapiens: string, Ailuropoda_melanoleuca: string ... 36 more fields],[Homo_sapiens: string, species: string ... 1 more field],[species: string, common

In [86]:
val ranked = spark.readTSV(ranked_path, true)
val top = ranked.select("symbol").as[String].collect.toList

In [87]:
File(intersections_path + "proteins").createDirectoryIfNotExists()
for((symbol, i) <- top.zipWithIndex)
{
    val folder_path = intersections_path + "proteins/"+i.toString+"_" + symbol
    File(folder_path).createDirectoryIfNotExists()
    val name = folder_path+"/"+symbol+".tsv"
    stage_two_exp.best_protein_by_symbol(symbol).writeTSV(name, header = true)
    println(s"${i} ${symbol} saved to ${name}")
}
//val noxa = stage_two_exp.best_protein_by_symbol("NOXA1")
//print(noxa.count)
//noxa.show(10)

parts of /data/sources/yspecies/data/output/intersections/proteins/0_NOXA1/NOXA1.tsv merged!
0 NOXA1 saved to /data/sources/yspecies/data/output/intersections/proteins/0_NOXA1/NOXA1.tsv
parts of /data/sources/yspecies/data/output/intersections/proteins/1_CEL/CEL.tsv merged!
1 CEL saved to /data/sources/yspecies/data/output/intersections/proteins/1_CEL/CEL.tsv
parts of /data/sources/yspecies/data/output/intersections/proteins/2_CALCOCO2/CALCOCO2.tsv merged!
2 CALCOCO2 saved to /data/sources/yspecies/data/output/intersections/proteins/2_CALCOCO2/CALCOCO2.tsv
parts of /data/sources/yspecies/data/output/intersections/proteins/3_C6orf89/C6orf89.tsv merged!
3 C6orf89 saved to /data/sources/yspecies/data/output/intersections/proteins/3_C6orf89/C6orf89.tsv
parts of /data/sources/yspecies/data/output/intersections/proteins/4_PPP1CA/PPP1CA.tsv merged!
4 PPP1CA saved to /data/sources/yspecies/data/output/intersections/proteins/4_PPP1CA/PPP1CA.tsv
parts of /data/sources/yspecies/data/output/inters